In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

sample = pd.read_csv('sample.csv')
train = pd.read_csv('train.csv')

In [2]:
features = np.ones((700,21))

In [3]:
train['x6'] = train['x1']**2
train['x7'] = train['x2']**2
train['x8'] = train['x3']**2
train['x9'] = train['x4']**2
train['x10'] = train['x5']**2

train['x11'] = np.exp(train['x1'])
train['x12'] = np.exp(train['x2'])
train['x13'] = np.exp(train['x3'])
train['x14'] = np.exp(train['x4'])
train['x15'] = np.exp(train['x5'])

train['x16'] = np.cos(train['x1'])
train['x17'] = np.cos(train['x2'])
train['x18'] = np.cos(train['x3'])
train['x19'] = np.cos(train['x4'])
train['x20'] = np.cos(train['x5'])

train['x21'] = 1

In [4]:
reg = sm.OLS(train['y'], exog=sm.add_constant(train[train.columns[2:23]])).fit()
reg.params

/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


x1       262.097164
x2      -384.341965
x3      1844.798154
x4      -321.679920
x5      2854.011842
x6     -1965.259375
x7        90.209699
x8     -3791.148766
x9       553.926346
x10    -4873.285572
x11     -257.362811
x12      378.058148
x13    -1838.468980
x14      323.077446
x15    -2846.464128
x16    -4215.319279
x17      584.284149
x18    -9369.808241
x19     1417.226506
x20   -12489.139473
x21    28316.402606
dtype: float64

In [5]:
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import Ridge
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression

myscorer = make_scorer(mean_squared_error, squared=False)

mylambda = [0.001,0.1,1,10,100]
newlambda = [0,0,0,0,0]
output=[0,0,0,0,0]

for i in range(0,50):

    for i in range (0,len(mylambda)):
        clf = Ridge(alpha=mylambda[i])
        scores = cross_val_score(clf, train[train.columns[2:23]], train['y'], cv=10, scoring=myscorer)
        output[i] = scores.mean()
        
    if output[0] == min(output):
        newlambda[0] = mylambda[0]
        newlambda[4] = mylambda[1]
        newlambda[1] = mylambda[0] + (1/4)*(mylambda[1]-mylambda[0])
        newlambda[2] = mylambda[0] + (2/4)*(mylambda[1]-mylambda[0])
        newlambda[3] = mylambda[0] + (3/4)*(mylambda[1]-mylambda[0])
        
    if output[1] == min(output):
        newlambda[0] = mylambda[0]
        newlambda[4] = mylambda[2]
        newlambda[1] = mylambda[0] + (1/4)*(mylambda[2]-mylambda[0])
        newlambda[2] = mylambda[0] + (2/4)*(mylambda[2]-mylambda[0])
        newlambda[3] = mylambda[0] + (3/4)*(mylambda[2]-mylambda[0])
        
    if output[2] == min(output):
        newlambda[0] = mylambda[1]
        newlambda[4] = mylambda[3]
        newlambda[1] = mylambda[1] + (1/4)*(mylambda[3]-mylambda[1])
        newlambda[2] = mylambda[1] + (2/4)*(mylambda[3]-mylambda[1])
        newlambda[3] = mylambda[1] + (3/4)*(mylambda[3]-mylambda[1])
    
    if output[3] == min(output):
        newlambda[0] = mylambda[2]
        newlambda[4] = mylambda[4]
        newlambda[1] = mylambda[2] + (1/4)*(mylambda[4]-mylambda[2])
        newlambda[2] = mylambda[2] + (2/4)*(mylambda[4]-mylambda[2])
        newlambda[3] = mylambda[2] + (3/4)*(mylambda[4]-mylambda[2])
        
    if output[4] == min(output):
        newlambda[0] = mylambda[3]
        newlambda[4] = mylambda[4]
        newlambda[1] = mylambda[3] + (1/4)*(mylambda[3]-mylambda[1])
        newlambda[2] = mylambda[3] + (2/4)*(mylambda[3]-mylambda[1])
        newlambda[3] = mylambda[3] + (3/4)*(mylambda[3]-mylambda[1])
        
    for i in range(0,5) :
        mylambda[i] = newlambda[i]
        
print(mylambda)


[20.691965855795424, 20.691965855795466, 20.69196585579551, 20.691965855795555, 20.691965855795598]


In [9]:
clf = Ridge(alpha=mylambda[3], max_iter=100000)
clf.fit(train[train.columns[2:23]], train['y'])
print(clf.coef_)

from numpy import asarray
from numpy import savetxt

savetxt('save.csv', clf.coef_)

[ 0.32537425 -0.25353647  0.3501765   0.97372814  0.22806434 -0.18480556
 -0.18068442 -0.0581307  -0.24345183  0.02104711  0.23376967 -0.33881649
  0.3218634   0.88219612  0.23545529  0.09143509  0.08952446  0.02902446
  0.11982519 -0.01005267  0.        ]
